In [1]:
import (
	"fmt"
	"github.com/kniren/gota/dataframe"
	"io/ioutil"
	"strconv"
	"strings"
)

In [2]:
type Pair struct {
	Phrase string
	Frequency int
}

In [3]:
func pairsAndFilters(splitPairs []string) ([]Pair, map[string]bool) {
	var (
		pairs []Pair
		m map[string]bool
	)
	m = make(map[string]bool)
	for _, pair := range splitPairs {
		p := strings.Split(pair, ":")
		phrase := p[0]
		m[phrase] = true
		if len(p) < 2 {
			continue
		}
		freq, err := strconv.Atoi(p[1])
		if err != nil {
			continue
		}
		pairs = append(pairs, Pair{
			Phrase: phrase,
			Frequency: freq,
		})
	}
	return pairs, m
}


In [4]:
func exclude(pairs []Pair, exclusions map[string]bool) []Pair {
	var ret []Pair
	for i := range pairs {
		if !exclusions[pairs[i].Phrase] {
			ret = append(ret, pairs[i])
		}
	}
	return ret
}

In [5]:
	const kitchenReviews = "./datasets/words/kitchen"
	positives, err := ioutil.ReadFile(kitchenReviews + "/positive.review")
	negatives, err2 := ioutil.ReadFile(kitchenReviews + "/negative.review")
	if err != nil || err2 != nil {
		fmt.Println("Error(s)", err, err2)
	}

	pairsPositive := strings.Fields(string(positives))
	pairsNegative := strings.Fields(string(negatives))

	parsedPositives, posPhrases := pairsAndFilters(pairsPositive)
	parsedNegatives, negPhrases := pairsAndFilters(pairsNegative)
	parsedPositives = exclude(parsedPositives, negPhrases)
	parsedNegatives = exclude(parsedNegatives, posPhrases)

	dfPos := dataframe.LoadStructs(parsedPositives)
	dfNeg := dataframe.LoadStructs(parsedNegatives)

	dfPos = dfPos.Arrange(dataframe.RevSort("Frequency"))
	dfNeg = dfNeg.Arrange(dataframe.RevSort("Frequency"))

	fmt.Println(dfPos)
	fmt.Println(dfNeg)


[46383x2] DataFrame

    Phrase       Frequency
 0: tic-tac-toe  10       
 1: wusthoff     7        
 2: emperor      7        
 3: shot_glasses 6        
 4: pulp         6        
 5: games        6        
 6: sentry       6        
 7: gravel       6        
 8: the_emperor  5        
 9: aebleskivers 5        
    ...          ...      
    <string>     <int>    

[45760x2] DataFrame

    Phrase          Frequency
 0: seeds           9        
 1: perculator      7        
 2: probes          7        
 3: cork            7        
 4: coffee_tank     5        
 5: brookstone      5        
 6: convection_oven 5        
 7: black_goo       5        
 8: waring_pro      5        
 9: packs           5        
    ...             ...      
    <string>        <int>    



412 <nil>

In [6]:
import "bytes"
path := "./datasets/bmi/500_Person_Gender_Height_Weight_Index.csv"
b, err := ioutil.ReadFile(path)
if err != nil {
    fmt.Println("Error!", err)
}
df := dataframe.ReadCSV(bytes.NewReader(b))

In [7]:
df

[500x4] DataFrame

    Gender   Height Weight Index
 0: Male     174    96     4    
 1: Male     189    87     2    
 2: Female   185    110    4    
 3: Female   195    104    3    
 4: Male     149    61     3    
 5: Male     189    104    3    
 6: Male     147    92     5    
 7: Male     154    111    5    
 8: Male     174    90     3    
 9: Female   169    103    4    
    ...      ...    ...    ...  
    <string> <int>  <int>  <int>


In [8]:
fmt.Println("Minimum", df.Col("Height").Min())
fmt.Println("Maximum", df.Col("Height").Max())
fmt.Println("Mean", df.Col("Height").Mean())
fmt.Println("Median", df.Col("Height").Quantile(0.5))

Minimum 140
Maximum 199
Mean 169.944
Median 170


11 <nil>

In [9]:
df.Describe()

[7x5] DataFrame

    column   Gender   Height     Weight     Index   
 0: mean     -        169.944000 106.000000 3.748000
 1: stddev   -        16.375261  32.382607  1.355053
 2: min      Female   140.000000 50.000000  0.000000
 3: 25%      -        156.000000 80.000000  3.000000
 4: 50%      -        170.000000 106.000000 4.000000
 5: 75%      -        184.000000 136.000000 5.000000
 6: max      Male     199.000000 160.000000 5.000000
    <string> <string> <float>    <float>    <float> 


In [10]:
import "gonum.org/v1/plot/plotter"
import "gonum.org/v1/plot"
import "gonum.org/v1/plot/vg"
import "bufio"

In [11]:
func SeriesToPlotValues(df dataframe.DataFrame, col string) plotter.Values {
	rows, _ := df.Dims()
	v := make(plotter.Values, rows)
	s := df.Col(col)
	for i := 0; i < rows; i++ {
	v[i] = s.Elem(i).Float()
	}
	return v
}

func HistogramData(v plotter.Values, title string) []byte {
	// Make a plot and set its title.
	p, err := plot.New()
	if err != nil {
		panic(err)
	}
	p.Title.Text = title
	h, err := plotter.NewHist(v, 10)
	if err != nil {
		panic(err)
	}
	//h.Normalize(1) // Uncomment to normalize the area under the histogram to 1
	p.Add(h)
	w, err := p.WriterTo(5*vg.Inch, 4*vg.Inch, "jpg")
	if err != nil {
		panic(err)
	}
	var b bytes.Buffer
	writer := bufio.NewWriter(&b)
	w.WriteTo(writer)
	return b.Bytes()
}


In [12]:
display.JPEG(HistogramData(SeriesToPlotValues(df, "Height"), "Height Histogram"))